# Introduction
Group members(& emails?):

# Table of contents
TODO: titles with internal links to move there

# Decomposition and breakdown

blah blah

# Analysis of data given (records.csv)

# Planning and design of algorithm
insert diagram drawn by Cherry here

## Flowchart and pseudocode

lorem ipsum

# Excel things

In [ ]:
'''
file = original excel file
rawFileData = giant list where each item is a row converted into a single string
fileHeader = first row "Tutorial Group,Student ID,School,Name,Gender,CGPA"
fileData = each row in the file turned into a list of cells as strings
studentData = list of dictionaries, each representing a student's data ('student ID', 'TG', 'School', 'Name', 'Gender', 'CGPA')

In [39]:
file = open("records.csv", "r")
rawFileData = file.readlines()
fileData = []
for i in rawFileData:
    i = i.split(",")
    i[5] = i[5].replace('\n', '')
    fileData.append(i)
fileHeader = fileData.pop(0) #list of cells from first line

print(fileData[0])

['G-1', '5002', 'CCDS', 'Aarav Singh', 'Male', '4.02']


In [26]:
#global variables


In [27]:
def initStudents():      #creates studentData, a list of dictionaries, each representing a student
    studentData = []
    studentCount = 0
    for row in fileData:
        studentCount += 1
        if row[4] == 'Male':
            gender = True
        elif row[4] == 'Female':
            gender = False
        studentData.append({'Student ID': int(row[1]),'TG': row[0], 'School': row[2], 'Name': row[3], 'Gender': gender, 'CGPA': float(row[5])})
    return studentData

In [28]:
studentData = initStudents()
print(studentData[0])


{'Student ID': 5002, 'TG': 'G-1', 'School': 'CCDS', 'Name': 'Aarav Singh', 'Gender': True, 'CGPA': 4.02}


# Algo for sorting

# Getting it back into an excel file and things

In [40]:
testfile = open("testrecords.csv", "w")
fileHeader = fileHeader
fileData.insert(0, fileHeader) #puts back header line
fileData[1].append('')
fileData[1].append('AAAAAAAH')
print(fileData[0])
print(fileData[1])
for i in range(len(fileData)):  #for each list, join items by ','. Then prints each line into new excel sheet
    fileData[i] = ','.join(fileData[i])
    print(fileData[i], file = testfile, end = '\n')


['Tutorial Group', 'Student ID', 'School', 'Name', 'Gender', 'CGPA']
['G-1', '5002', 'CCDS', 'Aarav Singh', 'Male', '4.02', '', 'AAAAAAAH']


# Algorithm Evaluation

## Basic Factors
No majority of students of the same following factors:
1. School
2. Gender
3. Current CGPA

General flow:
TODO: insert flowchart
- For each tutorial group, check for a single factor first, to find the group with the least diversity
- Then rank them to see with tutorial group has a group with the least diversity

### Finding the diversity of a single factor of a single group
Required data: list of students in the group, required factor of the student
<br>
Output: The number for diversity
<br>
Start with finding the number of occurences of each factor

In [1]:
# Parameters:
#  factor: string of factor to check, same as the key in the dictionary
#  groupData: data needed as a list of dictionaries
# Return:
#  occurences: dictionary of {factor1: numOccurences, factor2: numOccurences ... } as string:int
def FindOccurencesInGroup(factor, groupData : list) -> dict:
    size = len(groupData)
    occurences = {} # Dictionary to keep track of occurences of each factor
    for i in groupData:
        for studentData in i.values():
            schoolOfStudent = studentData[factor]
            # If factor already exists in occurences, add 1 to it, otherwise create it and set to 1
            if schoolOfStudent in occurences.keys():
                occurences[schoolOfStudent] += 1
            else:
                occurences[schoolOfStudent] = 1
    return occurences

In [2]:
testGroupData = [
    {71 : {"TG" : "G-1", "School" : "SoH", "Name" : "Savannah Taylor", "Gender" : False, "CGPA" : 4.19}},
    {75 : {"TG" : "G-1", "School" : "CCDS", "Name" : "Sakina Ahmed", "Gender" : False, "CGPA" : 4.03}},
    {162 : {"TG" : "G-1", "School" : "SSS", "Name" : "Lila Patel", "Gender" : False, "CGPA" : 4.07}},
    {235 : {"TG" : "G-1", "School" : "CCDS", "Name" : "Ming Zhang", "Gender" : True, "CGPA" : 4.06}},
    {288 : {"TG" : "G-1", "School" : "CoB (NBS)", "Name" : "Ajay Verma", "Gender" : True, "CGPA" : 4.01}}
]
# testGroupData = [
#     {"Student ID": 71, "TG" : "G-1", "School" : "SoH", "Name" : "Savannah Taylor", "Gender" : False, "CGPA" : 4.19},
#     {"Student ID": 75, "TG" : "G-1", "School" : "CCDS", "Name" : "Sakina Ahmed", "Gender" : False, "CGPA" : 4.03},
#     {"Student ID": 162, "TG" : "G-1", "School" : "SSS", "Name" : "Lila Patel", "Gender" : False, "CGPA" : 4.07},
#     {"Student ID": 235, "TG" : "G-1", "School" : "CCDS", "Name" : "Ming Zhang", "Gender" : True, "CGPA" : 4.06},
#     {"Student ID": 288, "TG" : "G-1", "School" : "CoB (NBS)", "Name" : "Ajay Verma", "Gender" : True, "CGPA" : 4.01}
# ]
print(FindOccurencesInGroup("School", testGroupData))
print(FindOccurencesInGroup("Gender", testGroupData))
print(FindOccurencesInGroup("CGPA", testGroupData))

{'SoH': 1, 'CCDS': 2, 'SSS': 1, 'CoB (NBS)': 1}
{False: 3, True: 2}
{4.19: 1, 4.03: 1, 4.07: 1, 4.06: 1, 4.01: 1}


#### Then find the most number of occurences out of the total students

In [3]:
# Summary: returns the key and value of the key with the largest value
# Parameters:
#  dictionary: the dict to find the max of
# Return:
#  key : list, values : int, where key is the key with highest value
def FindMaxInDict(dictionary: dict) -> (list, int):
    maxKeys = []
    maxVal = 0
    for key, value in dictionary.items():
        # update maxKeys and maxVal whenever a new value >= current maxVal
        if value > maxVal:
            maxKeys = [key]
            maxVal = value
        elif value == maxVal:
            maxKeys.append(key)
    return maxKeys, maxVal

In [4]:
print(FindMaxInDict(FindOccurencesInGroup("School", testGroupData)))
print(FindMaxInDict(FindOccurencesInGroup("Gender", testGroupData)))
print(FindMaxInDict(FindOccurencesInGroup("CGPA", testGroupData)))

(['CCDS'], 2)
([False], 3)
([4.19, 4.03, 4.07, 4.06, 4.01], 1)


## CGPA case
But this does not work for CGPA, for that we want to find the mean, range, and midrange instead
If we use only mean, a majority of high CGPA with a very low CGPA will result in a medium CGPA, this can be solved by looking at midrange, where a high difference between midrange and mean will show that this is happening
However, even with these two, a group will all medium CGPAs will also result in medium mean, and low midrange difference, therefore we add range to determine that this is happening if the range is small.
In conclusion, the result we are looking for is one with an medium mean, high range, and low difference between midrange and mean.
### Finding mean, midrange, and range

In [5]:
def Mean(*nums) -> float:
    if len(nums) == 1 and type(nums[0]) is list:
        nums = nums[0]
    return sum(nums) / len(nums)

def Range(*nums) -> float:
    if len(nums) == 1 and type(nums[0]) is list:
        nums = nums[0]
    return max(nums) - min(nums)

def Midrange(*nums) -> float:
    if len(nums) == 1 and type(nums[0]) is list:
        nums = nums[0]
    return (max(nums) + min(nums)) / 2

In [6]:
testList = [4.19, 4.03, 4.07, 4.06, 4.01]
print(Mean(testList))
print(Range(testList))
print(Midrange(testList))

4.072
0.1800000000000006
4.1


## Calculating the factors for each group

In [ ]:
# Parameters:
#  factors: list of strings of factor to check, same as the key in the dictionary
#  groupDatas: List of datas needed as a list of dictionaries
# Return:
#  leastDiverse: dictionary of {factor1: numOccurences, factor2: numOccurences ... } as string:int
def FindLeastDiverseInTutorialGroup(factors, groupDatas) -> dict:
    leastDiverse = {}
    

In [ ]:
# Weights
CRITERA_WEIGHTS = {"CGPA": 100, "SCHOOL": 100, "GENDER": 100}

# Graphical plots and things

# Challenges and Issues

# Conclusion

# Debug test things or test cases?
doubt they will give marks for test cases tho, prob just leave this for our own use and remove before submission
Highlight